In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
from sklearn.metrics import accuracy_score , confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import theano.tensor as tt
import arviz as az

In [ ]:
data = pd.read_csv("balanced_sample.csv")
#For training and testing only on the small df
#feature_columns = ['BEN','EBE', 'CO', 'NMHC', 'NO_2', 'O_3', 'PM10', 'PM25', 'SO_2','TCH','TOL'] #11 features - Best
feature_columns = ["NO_2", "O_3", "PM10", "PM25", "SO_2"] #testing with less 
X = data[feature_columns].values

# Target variable y
target_column = "AQI_GenPop_Index" 
y = data[target_column].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_features = X_train.shape[1]
n_classes=2

In [ ]:
with pm.Model() as AQI_model:
    # Priors for coefficients and bias, with better starting values
    coeffs = pm.Normal("coeffs", mu=0, sigma=1, shape=n_features, testval=np.zeros((n_features)))
    bias = pm.Normal("bias", mu=0, sigma=1)

def logistic(x, epsilon=1e-6):
    return 1 / (1 + tt.exp(-x)) + epsilon
  
    p = logistic(pm.math.dot(X_train, coeffs) + bias)
    # Define the Bernoulli likelihood
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y_train)    
# MCMC
with AQI_model:
    #step=pm.Metropolis()
    #step=pm.NUTS(target_accept=0.8)
    #n_init only for advi methods, default 20000
    trace = pm.sample(2000, tune=1600, chains=8, cores=4, return_inferencedata=False )#,step=step, 
    sns.set_palette("BuPu")
    pm.plot_trace(trace)

az.summary(trace)
sns.set_palette("BuPu_r")
with AQI_model:
    az.plot_trace(trace, compact=False)

In [ ]:
# BINARY Predicting on test data
def predict_proba(X, trace):
    linear = np.dot(X, trace["coeffs"].mean(axis=0)) + trace["bias"].mean()
    proba = 1 / (1 + np.exp(-linear))
    return np.column_stack((1 - proba, proba)) #(proba negative(0), proba positve(1))
y_test_pred_proba = predict_proba(X_test, trace)
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

In [ ]:
# Evaluation of model performance
accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:", accuracy)

from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_test_pred, average='macro') #try also 'micro' or 'weighted'
print("Precision:", precision)
print("Recall:", recall)

from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test, y_test_pred)
print('ROC AUC:', roc_score) #roc_score<0.5-->poor classifier, =0.5-->random classifier, 
                             #>0.7-->good classifier, 0.8->strong, 1->best